In [114]:
import pandas as pd
import numpy as np

import plotly.express as px

from math import radians, sin, cos, sqrt, atan2

from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans, DBSCAN, MeanShift, estimate_bandwidth
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_score

In [115]:
df = pd.read_csv("src/fraud test.csv")
df.describe(include="all")

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
count,555719.000000,555719,5.557190e+05,555719,555719,555719.000000,555719,555719,555719,555719,...,555719.000000,555719.000000,5.557190e+05,555719,555719,555719,5.557190e+05,555719.000000,555719.000000,555719.000000
unique,NaN,226976,NaN,693,14,NaN,341,471,2,924,...,NaN,NaN,NaN,478,910,555719,NaN,NaN,NaN,NaN
top,NaN,15/12/2020 21:26,NaN,fraud_Kilback LLC,gas_transport,NaN,Christopher,Smith,F,444 Robert Mews,...,NaN,NaN,NaN,Film/video editor,23/03/1977,2da90c7d74bd46a0caf3777415b3ebd3,NaN,NaN,NaN,NaN
freq,NaN,16,NaN,1859,56370,NaN,11443,12146,304886,1474,...,NaN,NaN,NaN,4119,2408,1,NaN,NaN,NaN,NaN
mean,277859.000000,NaN,4.178387e+17,NaN,NaN,69.392810,NaN,NaN,NaN,NaN,...,38.543253,-90.231325,8.822189e+04,NaN,NaN,NaN,1.380679e+09,38.542798,-90.231380,0.003860
std,160422.401459,NaN,1.309837e+18,NaN,NaN,156.745941,NaN,NaN,NaN,NaN,...,5.061336,13.721780,3.003909e+05,NaN,NaN,NaN,5.201104e+06,5.095829,13.733071,0.062008
min,0.000000,NaN,6.041621e+10,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,...,20.027100,-165.672300,2.300000e+01,NaN,NaN,NaN,1.371817e+09,19.027422,-166.671575,0.000000
25%,138929.500000,NaN,1.800430e+14,NaN,NaN,9.630000,NaN,NaN,NaN,NaN,...,34.668900,-96.798000,7.410000e+02,NaN,NaN,NaN,1.376029e+09,34.755302,-96.905129,0.000000
50%,277859.000000,NaN,3.521420e+15,NaN,NaN,47.290000,NaN,NaN,NaN,NaN,...,39.371600,-87.476900,2.408000e+03,NaN,NaN,NaN,1.380762e+09,39.376593,-87.445204,0.000000
75%,416788.500000,NaN,4.635330e+15,NaN,NaN,83.010000,NaN,NaN,NaN,NaN,...,41.894800,-80.175200,1.968500e+04,NaN,NaN,NaN,1.385867e+09,41.954163,-80.264637,0.000000


In [116]:
# Séparation des données en deux dataset selon le coté frauduleux de la transaction
df_frauds = df[df['is_fraud'] == 1]
sample = df[df['is_fraud'] == 0]

# Échantillonnage du dataset de transactions non frauduleuses pour avoir la même taille que le dataset de transactions frauduleuses
dsam = sample.sample(2145, random_state=0)

# Fusion des deux datasets
frames = [df_frauds, dsam]
result = pd.concat(frames)

# Mélange des données dans le dataset
result = result.sample(frac = 1)
result.shape

(4290, 23)

In [117]:
result.describe(include="all")

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
count,4290.000000,4290,4.290000e+03,4290,4290,4290.000000,4290,4290,4290,4290,...,4290.000000,4290.000000,4.290000e+03,4290,4290,4290,4.290000e+03,4290.000000,4290.000000,4290.000000
unique,NaN,4237,NaN,680,14,NaN,309,430,2,798,...,NaN,NaN,NaN,445,786,4290,NaN,NaN,NaN,NaN
top,NaN,30/06/2020 23:16,NaN,fraud_Kilback LLC,grocery_pos,NaN,Christopher,Williams,F,444 Robert Mews,...,NaN,NaN,NaN,Science writer,22/09/1997,c20cca6d1f922312cf89f40bfdae6c5d,NaN,NaN,NaN,NaN
freq,NaN,3,NaN,26,693,NaN,119,106,2352,24,...,NaN,NaN,NaN,44,24,1,NaN,NaN,NaN,NaN
mean,261787.030070,NaN,3.810071e+17,NaN,NaN,299.209186,NaN,NaN,NaN,NaN,...,38.783911,-90.376413,8.239483e+04,NaN,NaN,NaN,1.380186e+09,38.784218,-90.380031,0.500000
std,149535.273281,NaN,1.261528e+18,NaN,NaN,374.387217,NaN,NaN,NaN,NaN,...,5.025407,13.864927,2.764307e+05,NaN,NaN,NaN,4.910323e+06,5.052313,13.871188,0.500058
min,370.000000,NaN,6.041621e+10,NaN,NaN,1.020000,NaN,NaN,NaN,NaN,...,20.027100,-165.672300,2.300000e+01,NaN,NaN,NaN,1.371824e+09,19.161782,-166.033127,0.000000
25%,135552.000000,NaN,3.885950e+13,NaN,NaN,20.390000,NaN,NaN,NaN,NaN,...,34.929800,-96.745600,8.610000e+02,NaN,NaN,NaN,1.375913e+09,34.975099,-96.777858,0.000000
50%,259887.000000,NaN,3.500170e+15,NaN,NaN,88.580000,NaN,NaN,NaN,NaN,...,39.599400,-87.823500,2.676000e+03,NaN,NaN,NaN,1.380066e+09,39.589137,-87.703727,0.500000
75%,385890.500000,NaN,4.452370e+15,NaN,NaN,467.127500,NaN,NaN,NaN,NaN,...,42.076500,-80.158000,1.564700e+04,NaN,NaN,NaN,1.384815e+09,42.049737,-80.108471,1.000000


In [118]:
# Définition des colonnes à enlever pour le modèle de clustering
to_drop = ['cc_num', 'first', 'last', 'street', 'city' ,'state', 'zip', 'trans_num']
clean_data = result.drop(columns=to_drop)

In [119]:
# Définition de la fonction de calcul de la distance
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Rayon de la Terre en kilomètres

    # Convertir les latitudes et longitudes en radians
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # Différences de latitude et longitude
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Formule haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    # Distance en kilomètres
    distance = R * c
    
    return distance
    # Assurez-vous que cette fonction est correctement indentée ici

# Calcul de la distance pour chaque ligne du DataFrame
clean_data['distance'] = clean_data.apply(lambda row: haversine_distance(row['lat'], row['long'], row['merch_lat'], row['merch_long']), axis=1)

# Affichage des premières lignes du DataFrame avec la nouvelle colonne
clean_data.head()

,Unnamed: 0,trans_date_trans_time,merchant,category,amt,gender,lat,long,city_pop,job,dob,unix_time,merch_lat,merch_long,is_fraud,distance
499348,499348,19/12/2020 21:10,"fraud_Kling, Howe and Schneider",home,25.57,F,44.4530,-123.1065,917,Radio broadcast assistant,08/07/1995,1387487400,45.151878,-122.743978,0,82.807852
247192,247192,19/09/2020 17:37,fraud_Thompson-Gleason,health_fitness,28.64,M,39.8974,-82.4071,9051,Audiological scientist,08/01/1983,1379612267,39.563607,-83.146897,0,73.347762
70676,70676,15/07/2020 10:17,fraud_Pacocha-O'Reilly,grocery_pos,98.94,M,44.2378,-95.2739,1507,Land/geomatics surveyor,12/10/1993,1373883470,45.116957,-95.797124,0,106.150630
252420,252420,21/09/2020 16:10,fraud_Collier LLC,home,61.28,F,33.6773,-118.0051,190249,"Therapist, horticultural",17/09/1976,1379779836,34.290132,-118.543945,0,84.332084
143870,143870,10/08/2020 15:27,"fraud_Hills, Hegmann and Schaefer",health_fitness,22.80,M,38.4921,-85.4524,564,Pensions consultant,10/04/1996,1376148449,37.710637,-86.223115,0,109.992959


In [120]:
clean_data.describe(include="all")

,Unnamed: 0,trans_date_trans_time,merchant,category,amt,gender,lat,long,city_pop,job,dob,unix_time,merch_lat,merch_long,is_fraud,distance
count,4290.000000,4290,4290,4290,4290.000000,4290,4290.000000,4290.000000,4.290000e+03,4290,4290,4.290000e+03,4290.000000,4290.000000,4290.000000,4290.000000
unique,NaN,4237,680,14,NaN,2,NaN,NaN,NaN,445,786,NaN,NaN,NaN,NaN,NaN
top,NaN,30/06/2020 23:16,fraud_Kilback LLC,grocery_pos,NaN,F,NaN,NaN,NaN,Science writer,22/09/1997,NaN,NaN,NaN,NaN,NaN
freq,NaN,3,26,693,NaN,2352,NaN,NaN,NaN,44,24,NaN,NaN,NaN,NaN,NaN
mean,261787.030070,NaN,NaN,NaN,299.209186,NaN,38.783911,-90.376413,8.239483e+04,NaN,NaN,1.380186e+09,38.784218,-90.380031,0.500000,76.416120
std,149535.273281,NaN,NaN,NaN,374.387217,NaN,5.025407,13.864927,2.764307e+05,NaN,NaN,4.910323e+06,5.052313,13.871188,0.500058,29.209701
min,370.000000,NaN,NaN,NaN,1.020000,NaN,20.027100,-165.672300,2.300000e+01,NaN,NaN,1.371824e+09,19.161782,-166.033127,0.000000,1.959344
25%,135552.000000,NaN,NaN,NaN,20.390000,NaN,34.929800,-96.745600,8.610000e+02,NaN,NaN,1.375913e+09,34.975099,-96.777858,0.000000,56.089232
50%,259887.000000,NaN,NaN,NaN,88.580000,NaN,39.599400,-87.823500,2.676000e+03,NaN,NaN,1.380066e+09,39.589137,-87.703727,0.500000,78.884395
75%,385890.500000,NaN,NaN,NaN,467.127500,NaN,42.076500,-80.158000,1.564700e+04,NaN,NaN,1.384815e+09,42.049737,-80.108471,1.000000,99.072109


In [121]:
mean_distance_fraudulent = clean_data.loc[clean_data["is_fraud"] == 1, "distance"].mean()

print("Moyenne de la distance pour les transactions frauduleuses :", mean_distance_fraudulent)

Moyenne de la distance pour les transactions frauduleuses : 76.21390498710234


In [122]:
# Sélection des features nécessaires pour le clustering
data_geo = clean_data[["merch_lat", "merch_long", "is_fraud"]]

In [123]:
# Première répartition des transactions

# Créer une palette de couleurs personnalisée avec le rouge pour les valeurs de is_fraud égales à 1
color_scale = px.colors.diverging.RdYlBu_r

# Modifier la couleur pour les valeurs de is_fraud égales à 1 en rouge
color_scale[0] = 'blue'

# Définir le centre de la carte et le niveau de zoom pour afficher toutes les données
center_longitude = data_geo['merch_long'].mean()  # Utiliser la moyenne des longitudes comme centre
center_latitude = data_geo['merch_lat'].mean()  # Utiliser la moyenne des latitudes comme centre
zoom_level = 2  # Niveau de zoom pour afficher toutes les données

fig = px.scatter_mapbox(
        data_geo,
        lat="merch_lat",
        lon="merch_long",
        color="is_fraud",
        mapbox_style="carto-positron",
        title="Répartition géographiques de l'échantillon des transactions",
        color_continuous_scale=color_scale,
        center=dict(lon=center_longitude, lat=center_latitude), 
        zoom=zoom_level
)

fig.show()

# Standardisation des données

In [124]:
num_feat = [0, 1, 2]
num_trans = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_trans, num_feat)])

X= preprocessor.fit_transform(data_geo)
print(X[0:5, :])

[[ 1.26049235 -2.33344972 -1.        ]
 [ 0.15428177  0.52151098 -1.        ]
 [ 1.25357966 -0.39057395 -1.        ]
 [-0.88961424 -2.03062616 -1.        ]
 [-0.21251773  0.29971479 -1.        ]]


In [125]:
df_fr = data_geo[data_geo['is_fraud']==1]

# Clustering des transactions frauduleuses
## Kmeans
### Silhouette

In [126]:
from sklearn.metrics import silhouette_score

sil = []
k = []

## Commencez à i=3 pour exclure n_clusters=2
for i in range(2, 31):
    kmeans = KMeans(n_clusters=i, random_state=0, n_init='auto')
    kmeans.fit(df_fr)
    sil.append(silhouette_score(df_fr, kmeans.labels_))
    k.append(i)

# Créer une data frame
cluster_scores = pd.DataFrame(sil)
k_frame = pd.Series(k)

# Créer une figure avec Plotly Express
fig = px.bar(data_frame=cluster_scores, x=k, y=cluster_scores.iloc[:, -1])

# Ajouter un titre et des étiquettes d'axe
fig.update_layout(
    yaxis_title="Silhouette Score",
    xaxis_title="# Clusters",
    title="Silhouette Score par cluster"
)

fig.show()  # Afficher la figure

On peut voir grâce à la méthode Silhouette que la répartition en clusters est le plus efficace sur les valeurs 3, 7 et 24 en nombres de clusters

### ELBOW

In [127]:
# Instanciate KMeans with k=3 and initialisation with k-means++
# You should always use k-means++ as it alleviate the problem of local minimum convergence
kmeans = KMeans(n_clusters=10, random_state=0)

# Fit kmeans to our dataset
kmeans.fit(df_fr)
# Let's create a loop that will collect the Within-sum-of-square (wcss) for each value K
# Let's use .inertia_ parameter to get the within sum of square value for each value K
wcss =  []
k = []
for i in range (1,21):
    kmeans = KMeans(n_clusters= i, random_state = 0, n_init = 'auto')
    kmeans.fit(df_fr)
    wcss.append(kmeans.inertia_)
    k.append(i)
# Create DataFrame
wcss_frame = pd.DataFrame(wcss)
k_frame = pd.Series(k)

# Create figure
fig= px.line(
    wcss_frame,
    x=k_frame,
    y=wcss_frame.iloc[:,-1]
)

# Create title and axis labels
fig.update_layout(
    yaxis_title="Inertia",
    xaxis_title="# Clusters",
    title="Inertia per cluster"
)

# Render
#fig.show(renderer="notebook")
fig.show() # if using workspace

In [128]:
kmeans = KMeans(n_clusters=8, random_state=0)
kmeans.fit(df_fr)

# Prédire les clusters pour chaque échantillon
df_fr['cluster'] = kmeans.labels_

# Définir le centre de la carte et le niveau de zoom pour afficher toutes les données
center_longitude = df_fr['merch_long'].mean()  # Utiliser la moyenne des longitudes comme centre
center_latitude = df_fr['merch_lat'].mean()  # Utiliser la moyenne des latitudes comme centre
zoom_level = 2  # Niveau de zoom pour afficher toutes les données

# Créer une carte interactive avec Plotly Express en spécifiant le centre et le zoom
fig = px.scatter_mapbox(df_fr, lon='merch_long', lat='merch_lat', color='cluster', 
                        mapbox_style="carto-positron", title='Clusters des données géographiques selon la méthode Elbow',
                        center=dict(lon=center_longitude, lat=center_latitude), zoom=zoom_level)

# Afficher la carte interactive
fig.show()

C:\Users\mouto\AppData\Local\Temp\ipykernel_12260\3378152958.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



On peut voir grâce à la méthode Elbow, que la répartition en cluster n'est pas optimale et peut s'avérer anarchique. Bien que les zones de villes ne soient découpées entre les deux clusters, cette méthode manque de précision

### DBSCAN

In [129]:
eps = 1
min_samples = 3
p = 1.5

In [130]:
db = DBSCAN(eps=eps, min_samples=min_samples, metric="minkowski", p=p, algorithm='brute')
db.fit(df_fr)
display(np.unique(db.labels_, return_counts=True))

# Créer une copie du DataFrame si nécessaire
df_fr_copy = df_fr.copy()

# Convertir les étiquettes de cluster en int64
cluster_labels = db.labels_.astype('int64')

# Ajouter les labels de cluster à votre DataFrame en utilisant .loc
df_fr_copy.loc[:, 'cluster'] = cluster_labels

#
# Calculer le nombre d'éléments dans chaque cluster
cluster_sizes = df_fr_copy[df_fr_copy['cluster'] != -1]['cluster'].value_counts()

# Trier les clusters par ordre croissant de nombre d'éléments
sorted_clusters = cluster_sizes.sort_values()

# Créer une palette de couleurs divergente
colorscale = px.colors.diverging.balance[::-1]  # Utilisation d'une palette de couleurs divergente

# Mapper chaque cluster à une couleur en fonction de sa taille
cluster_color_mapping = {cluster: i for i, cluster in enumerate(sorted_clusters.index)}

# Ajouter les labels de cluster à votre DataFrame en utilisant .loc
df_fr_copy.loc[:, 'cluster'] = df_fr_copy['cluster'].map(cluster_color_mapping)

# Définir le centre de la carte et le niveau de zoom pour afficher toutes les données
center_longitude = df_fr_copy['merch_long'].mean()  # Utiliser la moyenne des longitudes comme centre
center_latitude = df_fr_copy['merch_lat'].mean()  # Utiliser la moyenne des latitudes comme centre
zoom_level = 2.5  # Niveau de zoom pour afficher toutes les données

# Créer une carte interactive avec Plotly Express en spécifiant le centre et le zoom
fig_fraud = px.scatter_mapbox(df_fr_copy[df_fr_copy['cluster'] != -1], lon='merch_long', lat='merch_lat', color='cluster', 
                        mapbox_style="carto-positron", title='Clusters des zones de transactions frauduleuses selon la méthode DBSCAN',
                        center=dict(lon=center_longitude, lat=center_latitude), zoom=zoom_level,
                        color_continuous_scale=colorscale)

# Afficher la carte interactive
fig_fraud.show()


(array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       dtype=int64),
 array([ 18, 611,  21, 138, 182,   7, 393,  14,   8, 257,  43,  28,   9,
         28,  36,  44,  46,  56,  71,  27,  16,  22,   3,   9,   5,   7,
          7,   3,  11,  14,  11], dtype=int64))

C:\Users\mouto\AppData\Local\Temp\ipykernel_12260\1882556984.py:12: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0  1  2 ... 15  0  3]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.

C:\Users\mouto\AppData\Local\Temp\ipykernel_12260\1882556984.py:28: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[29. 14. 25. ... 22. 29. 26.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



On peut voir, grâce au dbscan, que les transactions frauduleuses sont réparties principalement autour de grandes villes. Cependant la côte Est est beaucoup plus touchée par le phénomène que la côte Ouest.

### Mean Shift Clustering

In [131]:
# Estimation de la bande passante (bandwidth) automatique
bandwidth = estimate_bandwidth(df_fr, quantile=0.2, n_samples=10)

# Création du modèle Mean Shift
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)

# Entraînement du modèle
ms.fit(df_fr)

# Récupération des labels de cluster et des centres de cluster
labels = ms.labels_
cluster_centers = ms.cluster_centers_

# Nombre de clusters
n_clusters = len(cluster_centers)

print("Nombre de clusters:", n_clusters)

# Ajout des labels de cluster à votre DataFrame
df_fr_copy = df_fr.copy()
df_fr_copy['cluster'] = labels

# Calcul du nombre d'éléments dans chaque cluster
cluster_sizes = df_fr_copy[df_fr_copy['cluster'] != -1]['cluster'].value_counts()

# Tri des clusters par ordre croissant de nombre d'éléments
sorted_clusters = cluster_sizes.sort_values()

# Création d'une palette de couleurs divergente
colorscale = px.colors.diverging.balance[::-1]

# Mappage de chaque cluster à une couleur en fonction de sa taille
cluster_color_mapping = {cluster: i for i, cluster in enumerate(sorted_clusters.index)}

# Ajout des labels de cluster à votre DataFrame
df_fr_copy['cluster'] = df_fr_copy['cluster'].map(cluster_color_mapping)

# Définition du centre de la carte et du niveau de zoom pour afficher toutes les données
center_longitude = df_fr_copy['merch_long'].mean()
center_latitude = df_fr_copy['merch_lat'].mean()
zoom_level = 2.5

# Création d'une carte interactive avec Plotly Express
fig_fraud_ms = px.scatter_mapbox(df_fr_copy[df_fr_copy['cluster'] != -1], lon='merch_long', lat='merch_lat', color='cluster', 
                        mapbox_style="carto-positron", title='Clusters des zones de transactions frauduleuses selon la méthode Mean Shift',
                        center=dict(lon=center_longitude, lat=center_latitude), zoom=zoom_level,
                        color_continuous_scale=colorscale)

# Affichage de la carte interactive
fig_fraud_ms.show()


Nombre de clusters: 12


On peut remarquer que la méthode MSC créée les clusters de manières arbitraire. Cette méthode n'est donc pas retenue pour les résultats finaux

# Clustering avec l'ensemble des données

Maintenant que les clusters des transactions frauduleuses sont faites, nous nous attaquons aux données globables afin de pouvoir définir si les transactions frauduleuses suivent le schéma des transactions globales

In [132]:
clean_data.head()

,Unnamed: 0,trans_date_trans_time,merchant,category,amt,gender,lat,long,city_pop,job,dob,unix_time,merch_lat,merch_long,is_fraud,distance
499348,499348,19/12/2020 21:10,"fraud_Kling, Howe and Schneider",home,25.57,F,44.4530,-123.1065,917,Radio broadcast assistant,08/07/1995,1387487400,45.151878,-122.743978,0,82.807852
247192,247192,19/09/2020 17:37,fraud_Thompson-Gleason,health_fitness,28.64,M,39.8974,-82.4071,9051,Audiological scientist,08/01/1983,1379612267,39.563607,-83.146897,0,73.347762
70676,70676,15/07/2020 10:17,fraud_Pacocha-O'Reilly,grocery_pos,98.94,M,44.2378,-95.2739,1507,Land/geomatics surveyor,12/10/1993,1373883470,45.116957,-95.797124,0,106.150630
252420,252420,21/09/2020 16:10,fraud_Collier LLC,home,61.28,F,33.6773,-118.0051,190249,"Therapist, horticultural",17/09/1976,1379779836,34.290132,-118.543945,0,84.332084
143870,143870,10/08/2020 15:27,"fraud_Hills, Hegmann and Schaefer",health_fitness,22.80,M,38.4921,-85.4524,564,Pensions consultant,10/04/1996,1376148449,37.710637,-86.223115,0,109.992959


### KMeans

In [133]:
from sklearn.metrics import silhouette_score

sil = []
k = []

## Commencez à i=3 pour exclure n_clusters=2
for i in range(3, 31):
    kmeans = KMeans(n_clusters=i, random_state=0, n_init='auto')
    kmeans.fit(data_geo)
    sil.append(silhouette_score(data_geo, kmeans.labels_))
    k.append(i)

# Créer une data frame
cluster_scores = pd.DataFrame(sil)
k_frame = pd.Series(k)

# Créer une figure avec Plotly Express
fig = px.bar(data_frame=cluster_scores, x=k, y=cluster_scores.iloc[:, -1])

# Ajouter un titre et des étiquettes d'axe
fig.update_layout(
    yaxis_title="Silhouette Score",
    xaxis_title="# Clusters",
    title="Silhouette Score par cluster"
)

fig.show()  # Afficher la figure

In [134]:
# Instanciate KMeans with k=3 and initialisation with k-means++
# You should always use k-means++ as it alleviate the problem of local minimum convergence
kmeans = KMeans(n_clusters=10, random_state=0)

# Fit kmeans to our dataset
kmeans.fit(data_geo)
# Let's create a loop that will collect the Within-sum-of-square (wcss) for each value K
# Let's use .inertia_ parameter to get the within sum of square value for each value K
wcss =  []
k = []
for i in range (1,21):
    kmeans = KMeans(n_clusters= i, random_state = 0, n_init = 'auto')
    kmeans.fit(data_geo)
    wcss.append(kmeans.inertia_)
    k.append(i)
# Create DataFrame
wcss_frame = pd.DataFrame(wcss)
k_frame = pd.Series(k)

# Create figure
fig= px.line(
    wcss_frame,
    x=k_frame,
    y=wcss_frame.iloc[:,-1]
)

# Create title and axis labels
fig.update_layout(
    yaxis_title="Inertia",
    xaxis_title="# Clusters",
    title="Inertia per cluster"
)

# Render
#fig.show(renderer="notebook")
fig.show() # if using workspace

In [135]:
# Créer une copie du DataFrame si nécessaire
data_geo_copy = data_geo.copy()

# Créer une instance de KMeans et l'ajuster aux données
kmeans = KMeans(n_clusters=7, random_state=0)
kmeans.fit(data_geo_copy)

# Prédire les clusters pour chaque échantillon
data_geo_copy.loc[:, 'cluster'] = kmeans.labels_

# Définir le centre de la carte et le niveau de zoom pour afficher toutes les données
center_longitude = data_geo_copy['merch_long'].mean()  # Utiliser la moyenne des longitudes comme centre
center_latitude = data_geo_copy['merch_lat'].mean()  # Utiliser la moyenne des latitudes comme centre
zoom_level = 2  # Niveau de zoom pour afficher toutes les données

# Créer une carte interactive avec Plotly Express en spécifiant le centre et le zoom
fig = px.scatter_mapbox(data_geo_copy, lon='merch_long', lat='merch_lat', color='cluster', 
                        mapbox_style="carto-positron", title='Clusters des données géographiques selon la méthode KMeans',
                        center=dict(lon=center_longitude, lat=center_latitude), zoom=zoom_level)

# Afficher la carte interactive
fig.show()


# DBSCAN

In [136]:
eps = 0.7
min_samples = 2
p = 1.6

In [137]:
db = DBSCAN(eps=eps, min_samples=min_samples, metric="minkowski", p=p, algorithm='brute')
db.fit(data_geo)
display(np.unique(db.labels_, return_counts=True))

# Créer une copie du DataFrame si nécessaire
data_geo_copy = data_geo.copy()

# Convertir les étiquettes de cluster en int64
cluster_labels = db.labels_.astype('int64')

# Ajouter les labels de cluster à votre DataFrame en utilisant .loc
data_geo_copy.loc[:, 'cluster'] = cluster_labels

#
# Filtrer les données pour ne conserver que les points assignés à un cluster
data_geo_assigned = data_geo_copy[data_geo_copy['cluster'] != -1]

# Définir le centre de la carte et le niveau de zoom pour afficher toutes les données
center_longitude = data_geo_assigned['merch_long'].mean()
center_latitude = data_geo_assigned['merch_lat'].mean()
zoom_level = 2.5

# Spécifier une palette de couleurs qualitative avec plus de variations
colorscale = px.colors.qualitative.Dark24

# Créer une carte interactive avec Plotly Express en spécifiant le centre et le zoom
fig_all2 = px.scatter_mapbox(data_geo_assigned, lon='merch_long', lat='merch_lat', color='cluster', 
                        mapbox_style="carto-positron", title='Clusters des zones de transactions avec DBSCAN',
                        center=dict(lon=center_longitude, lat=center_latitude), zoom=zoom_level,
                        color_continuous_scale=colorscale)

# Afficher la carte interactive
fig_all2.show()

(array([ -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
         12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,
         25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
         38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
         51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
         64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
         77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
         90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
        103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
        116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
        129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
        142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
        155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173], dtype=int64)

# Mean Shift Clustering

In [148]:
from sklearn.cluster import MeanShift, estimate_bandwidth
import numpy as np
import pandas as pd
import plotly.express as px

# Estimation de la bande passante (bandwidth) automatique
bandwidth = estimate_bandwidth(data_geo, quantile=0.19, n_samples=12)

# Création du modèle Mean Shift
ms = MeanShift(bandwidth=bandwidth, bin_seeding=False)

# Entraînement du modèle
ms.fit(data_geo)

# Récupération des labels de cluster et des centres de cluster
labels = ms.labels_
cluster_centers = ms.cluster_centers_

# Nombre de clusters
n_clusters = len(cluster_centers)

print("Nombre de clusters:", n_clusters)

# Ajout des labels de cluster à votre DataFrame
data_geo_copy = data_geo.copy()
data_geo_copy['cluster'] = labels

# Calcul du nombre d'éléments dans chaque cluster
cluster_sizes = data_geo_copy[data_geo_copy['cluster'] != -1]['cluster'].value_counts()

# Tri des clusters par ordre croissant de nombre d'éléments
sorted_clusters = cluster_sizes.sort_values()

# Création d'une palette de couleurs divergente
colorscale = px.colors.diverging.balance[::-1]

# Mappage de chaque cluster à une couleur en fonction de sa taille
cluster_color_mapping = {cluster: i for i, cluster in enumerate(sorted_clusters.index)}

# Ajout des labels de cluster à votre DataFrame
data_geo_copy['cluster'] = data_geo_copy['cluster'].map(cluster_color_mapping)

# Définition du centre de la carte et du niveau de zoom pour afficher toutes les données
center_longitude = data_geo_copy['merch_long'].mean()
center_latitude = data_geo_copy['merch_lat'].mean()
zoom_level = 2.5

# Création d'une carte interactive avec Plotly Express
fig_fraud_ms = px.scatter_mapbox(data_geo_copy[data_geo_copy['cluster'] != -1], lon='merch_long', lat='merch_lat', color='cluster', 
                        mapbox_style="carto-positron", title='Clusters des zones de transactions selon la méthode Mean Shift',
                        center=dict(lon=center_longitude, lat=center_latitude), zoom=zoom_level,
                        color_continuous_scale=colorscale)

# Affichage de la carte interactive
fig_fraud_ms.show()


Nombre de clusters: 13


# Comparaison entre les clusters de fraudes et les clusters de transactions

In [139]:
display(fig_fraud.show())
display(fig_all2.show())

None

None

La méthode DBSCAN sur l'ensemble des données nous fait remarquer que les transactions frauduleuses suivent la même logique. Le DBSCAN sur les données globales arrive à retrouver similairement les mêmes clusters (soit le même groupe, soit par fusion de plusieurs clusters) même si on a plus de pertes par valeurs aberrantes.